In [1]:
import os
os.chdir('../../src/')
from visualization.SIR_Plot import make_SIR_graph, make_beds_graph

In [2]:
os.chdir('models')
import simulate_pandemic as simp
from actions import costs, city_restrictions
from MCFS import mcts, treeNode
from CMDP import CovidState

Loading Graph...Done!


In [3]:
from tqdm import tqdm
import numpy as np
import pickle as pkl
import datetime

In [4]:
pop_matrix = simp.init_infection()


In [5]:
pop_matrix.shape

(55492, 5)

In [6]:
import plotly.graph_objects as go
import pandas as pd

def make_beds_graph(data, actions, step_size, title, max_range=None, color_map=None, make_df=True):
    fig = go.Figure()
    
    color_map = {
        'Lockdown':          'rgb(0.83, 0.13, 0.15)',
        'Hard Quarantine':    'rgb(0.85, 0.35, 0.13)',
        'Light Quarantine':   'rgb(0.97, 0.91, 0.56)',
        'Social Distancing':  'rgb(0.67, 0.88, 0.69)',
        'Unrestricted':        'rgb(0.86, 0.86, 0.86)'    
    }

    pop = 55492
    
    actions = list(map(color_map.get,  actions))
    
    x = list(range(step_size*len(actions)))
    
    #\definecolor{royalblue(web)}{rgb}{0.25, 0.41, 0.88}
    #fig.add_trace(go.Scatter(x=x, y=data[3]/pop, name='hospitalized', line_color = 'rgb(0.25, 0.41, 0.88)',
    #                        line=dict(width=3.5),  
    #                        ))
    
    fig.add_trace(go.Scatter(x=x, y=(len(data))*[0.0015], name='capacity', line_color = 'black',
                            line=dict(dash='dash', width = 2),
                            ))
    # ACTIONS BACKGROUND################################################################################
    fig.update_layout(
        shapes=[
            dict(
                type="rect",
                # x-reference is assigned to the x-values
                xref="x",
                # y-reference is assigned to the plot paper [0,1]
                yref="paper",
                x0=(step_size*i),
                y0=0,
                x1=step_size*(i+1),
                y1=1,
                fillcolor=a,
                opacity=.7,
                layer="below",
                line_width=0,
            ) for i,a in enumerate(actions)]
    )

    fig.update_layout(xaxis={'showgrid': False,},
                      yaxis = {'showgrid': False, 'zeroline': False,},
                      showlegend=True, hovermode="x")
    
    fig.update_layout(
        xaxis = dict(
            tickmode = 'array',
            tickvals = list(range(0, len(data)+1, 7)),
            ticktext = list(range(0, int(len(data)/7) + 1, 1)),
            title = 'Plan Week',
            )
        )
    fig.update_layout(showlegend=False,
        plot_bgcolor='rgb(255,255,255)',
        margin=dict(
        l=50,
        r=50,
        b=50,
        t=50,
        pad=0
    ))
    
    fig.update_yaxes(automargin=True)
    #fig.write_image(f"{title}.pdf")

    
    return fig

In [7]:
pop_matrix = simp.init_infection()
actions=6*['Unrestricted']
days=7*len(actions)
step_size = 7
for day in tqdm(range(1, days+1)):
    #if less than 20% still susceptible, break simulation
    if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break            


    # Choose a new policy at each week
    if day % step_size == 1:             
        restrictions = city_restrictions[actions[int(day/7)]]

    pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
    pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
    pop_matrix = simp.update_population(pop_matrix)

#with open('pop_matrix.pkl', 'wb') as f:
#    pkl.dump(pop_matrix, f)
    
def simulate(pop_matrix, actions, step_size = 7, days = 364):
    data = []
    
    #with open('pop_matrix.pkl', 'rb') as f:
    #    pop_matrix = pkl.load(f)
    
    
    for day in tqdm(range(1, days+1)):
        #if less than 20% still susceptible, break simulation
        if pop_matrix[np.where(pop_matrix[:,1] == -1)].shape[0] > pop_matrix.shape[0]*.9: break            
            
        
        # Choose a new policy at each week
        if day % step_size == 1:
            restrictions = city_restrictions[actions[int(day/7)]]

        pop_matrix = simp.spread_infection(pop_matrix, restrictions, day)
        pop_matrix = simp.lambda_leak_expose(pop_matrix, day)
        pop_matrix = simp.update_population(pop_matrix)

        data.append(np.array(sorted(pop_matrix,key=lambda x: x[0]))[:,1]) 
    
    return data

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:02<00:00, 18.14it/s]


In [10]:
from joblib import Parallel, delayed

actions = (['Social Distancing']+['Light Quarantine'])
sims = Parallel(n_jobs=-1)(delayed(simulate)(pop_matrix=pop_matrix, actions=actions) 
                           for i in range(48))

In [11]:
for i in range(0, 29):
    fig = make_beds_graph(sims[0], actions, 7, '')
    for data in sims:
        data = pd.DataFrame([pd.Series(d).value_counts() for d in data])
        pop = data.iloc[0].sum()
        data=data[3].fillna(0)/pop
        x = list(range(i))
        fig.add_trace(go.Scatter(x=x, y=data.iloc[:i], name='hospitalized', mode='lines',
                                 line_color = 'rgb(0.25, 0.41, 0.88)', line=dict(width=2.5),
                                 opacity=0.85
                                ))
    fig.write_image(f"anims/sim_2_weeks_2_{i}.png", scale=3)
